# This notebook will take your regression or classification problem, the y variables (dependent) and x variables (independent) and determine which have what level of contribution to your dataset.
## This will allow you to visualize the information context of Xn to Y, and decide what to keep in future analyses, such as development of lienar regression

In [43]:
#Imports
import os
import glob as glob
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

import numpy as np
np.set_printoptions(precision=3, suppress=True)
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
#Calculate Correlation
from scipy.stats import pearsonr
import os
import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import platform

In [44]:
analysis = 'ols_linear_regression/default_analysis'
conn_path = r'/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/KiTH_Solutions/Research/Clinical Trial/study_metadata/age_interaction_glm.xlsx'
#----------------------------------------------------------------DONT TOUCH----------------------------------------------------------------
out_dir = os.path.join(conn_path.split('.')[0], f'{analysis}')
save = True
if os.path.exists(out_dir):
    pass
else:
    os.makedirs(out_dir)

## Import Pre-prepared Regression Data

## Prepare for regression

In [45]:
#----------------------------------------------------------------user input above----------------------------------------------------------------
from calvin_utils.file_utils.dataframe_utilities import remove_column_spaces
if os.path.basename(conn_path).split('.')[1] == 'csv':
    data_df = pd.read_csv(conn_path)
else:
    data_df = pd.read_excel(conn_path, sheet_name=0)
data_df = remove_column_spaces(data_df.reset_index(drop=True))
display(data_df)
# 
# display(data_df)

,Age,Type,Total
0,50,1,96
1,42,1,70
2,35,1,72
3,71,1,80
4,48,1,85
...,...,...,...
65,37,0,98
66,78,0,98
67,47,0,96
68,49,0,92


In [47]:
#Select specific subgroup
# outlier_index = (data_df['percent_change_adascog11'] <= -50)
# data_df = data_df.loc[outlier_index, :]

#Remove outlier
# outlier_index=[11, 47, 48, 49]
# data_df = data_df.drop(index=outlier_index)
# data_df.reset_index(drop=True, inplace=True)

try:
    data_df.pop('Patient_#_CDR,_ADAS')
    data_df.pop('Subiculum_CSF_Z_Score')
    data_df.pop('Subiculum')
    # data_df.pop('temporal')
    # data_df.pop('occipital')
    # data_df.pop('cerebellar')
    # data_df.pop('sbc_wm_zscores')
    # data_df.pop('left_hpc_wm_zscores')
    # data_df.pop('left_hpc_z_scores')	
    # data_df.pop('zscore_left_hpc_csf')	
    # data_df.pop('sbc_szcore')
    data_df.pop('ID')   
    data_df.pop('preOP')
    data_df.pop('postOP')
    data_df.pop('diff. Mattis')
    # data_df.pop('Age')
    data_df.pop('perc_change_mattis')
    data_df.pop('Unnamed: 0')
    # data_df.dropna(inplace=True)
    # data_df.reset_index(drop=True, inplace=True)

except:
    print('none to pop')

#move outcomes to start
# first_column = data_df.pop('Patient # CDR, ADAS')  # Remove column and store it in a variable
# data_df.insert(0, 'perc_improvement', first_column)  # Insert column at the first position


data_df.head(2)

none to pop


,Age,Type,Total
0,50,1,96
1,42,1,70


Rename Variables

In [48]:
#Rename the outcome variable
outcome_variable =  data_df.pop('%_Change_from_baseline_(ADAS-Cog11)')
data_df['outcome'] = outcome_variable

KeyError: '%_Change_from_baseline_(ADAS-Cog11)'

One-Hot Encode Specific Columns

In [49]:
data_df.columns

Index(['Age', 'Type', 'Total'], dtype='object')

In [50]:
# One-hot encode as needed
# data_df['interaction'] = data_df['Subiculum_Grey_Matter']*data_df['Subiculum_Connectivity']
data_df['Cognitive_Status'] = np.where(data_df['Cognitive_Status'] == 'MCI', 1, 0)
# data_df['Age'] = np.where(data_df['Age'] <= 65, 0, 1)

data_df


KeyError: 'Cognitive_Status'

Standardize

In [51]:
data_df.columns

Index(['Age', 'Type', 'Total'], dtype='object')

In [52]:
#Standardize the data
preserved_df = data_df.copy()
from sklearn.preprocessing import StandardScaler

# Remove anything you don't want to standardize
cols_not_to_standardize = ['Measurement', 'Subjective_Complaint', 'Attention_Complaint', 'Memory_Complaint',
       'Fluency_Complaint', 'Language_Complaint', 'Visuospatial_Complaint',
       'Impaired_on_ACE3', 'iADL_Impaired', 'ADL_Impaired']
# Select the columns to be standardized
try:
    cols_to_standardize = [col for col in data_df.columns if col not in cols_not_to_standardize]
except:
    print('No columns to standardize')
# Standardize
scaler = StandardScaler()
data_df[cols_to_standardize] = scaler.fit_transform(data_df[cols_to_standardize])




#Drop NAN
# data_df = data_df.dropna()
data_df

,Age,Type,Total
0,0.368183,1.0,0.982443
1,-0.264279,1.0,-1.283776
2,-0.817683,1.0,-1.109451
3,2.028396,1.0,-0.412153
4,0.210068,1.0,0.023658
...,...,...,...
65,-0.659567,-1.0,1.156768
66,2.581800,-1.0,1.156768
67,0.131010,-1.0,0.982443
68,0.289125,-1.0,0.633794


## Manipulate DF

Remove Rows

In [53]:
#Drop a specific set of rows
# data_df = data_df[data_df['Disease'] == 1]
data_df

,Age,Type,Total
0,0.368183,1.0,0.982443
1,-0.264279,1.0,-1.283776
2,-0.817683,1.0,-1.109451
3,2.028396,1.0,-0.412153
4,0.210068,1.0,0.023658
...,...,...,...
65,-0.659567,-1.0,1.156768
66,2.581800,-1.0,1.156768
67,0.131010,-1.0,0.982443
68,0.289125,-1.0,0.633794


remove columns

In [54]:
print(data_df.columns)


Index(['Age', 'Type', 'Total'], dtype='object')


In [55]:
# Tailor the Dataframe
data_df = data_df.loc[:, ['Age','Mesial_Temporal_Grade','Subiculum_Connectivity', 'outcome']]
data_df

KeyError: "['Mesial_Temporal_Grade', 'Subiculum_Connectivity', 'outcome'] not in index"

In [56]:
# over = data_df['basline'] >= np.mean(data_df['basline'])
# under = data_df['basline'] < np.mean(data_df['basline'])
# data_df['basline'][over] = 1
# data_df['basline'][under] = 0

# display(data_df)
# print(np.max(data_df.age))
mc_test = data_df.copy()
mc_test.pop('outcome')
# mc_test = mc_test.loc[:, ['Ventral_Attention', 'Limbic']]
# mc_test['interaction'] = mc_test['Limbic']*mc_test['Ventral_Attention']
from calvin_utils.statistical_utils.statistical_measurements import calculate_vif
calculate_vif(mc_test)

KeyError: 'outcome'

In [57]:
## GET INFORMATION ABOUT THE DATA
# sns.pairplot(data_df)
data_df.describe().transpose()


,count,mean,std,min,25%,50%,75%,max
Age,70.0,9.184122e-17,1.00722,-1.450145,-0.738625,-0.264279,0.348419,2.977089
Type,70.0,0.000000e+00,1.00722,-1.000000,-1.000000,0.000000,1.000000,1.000000
Total,70.0,6.740640e-17,1.00722,-2.678373,-0.673640,0.197983,0.873491,1.243930


## Generate Statsmodel Linear Model

In [58]:
print(data_df.columns)

Index(['Age', 'Type', 'Total'], dtype='object')


In [61]:
import statsmodels.formula.api as smf

formula = 'Total~Age*Type'
#----------------------------------------------------------------DO NOT TOUCH
results = smf.ols(formula, data=data_df).fit()
print(results.summary2())

                 Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.035   
Dependent Variable: Total            AIC:                201.0477
Date:               2023-08-07 10:48 BIC:                210.0417
No. Observations:   70               Log-Likelihood:     -96.524 
Df Model:           3                F-statistic:        1.834   
Df Residuals:       66               Prob (F-statistic): 0.150   
R-squared:          0.077            Scale:              0.97901 
------------------------------------------------------------------
                Coef.   Std.Err.     t     P>|t|    [0.025  0.975]
------------------------------------------------------------------
Intercept       0.0000    0.1183   0.0000  1.0000  -0.2361  0.2361
Age            -0.1809    0.1183  -1.5295  0.1309  -0.4170  0.0552
Type           -0.2079    0.1183  -1.7583  0.0833  -0.4441  0.0282
Age:Type       -0.0312    0.1183  -0.2637  0.7929  -0.2673  0.2049
--------------------

In [40]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.model_selection import LeaveOneOut
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error

#-----------------------------------------------------------------------------DO NOT TOUCH THIS!----------------------------------------------------------------
# Assuming data_df is already defined
loo = LeaveOneOut()
loo.get_n_splits(data_df)

models = []
squared_error_list = []
predictions = []
actual_values = []

for train_index, test_index in loo.split(data_df):
    train_data, test_data = data_df.iloc[train_index], data_df.iloc[test_index]

    model = smf.ols(f'{formula.split(" ~ ")[0]}'' ~ 'f'{formula.split(" ~ ")[1]}', data=train_data).fit()
    models.append(model)

    test_x = test_data.drop(columns=[f'{formula.split(" ~ ")[0]}'])
    test_y = test_data[f'{formula.split(" ~ ")[0]}']
    pred_y = model.predict(test_x)

    squared_error = np.square(pred_y - test_y)
    squared_error_list.extend(squared_error)

    predictions.extend(pred_y)
    actual_values.extend(test_y)

average_squared_error = np.mean(squared_error_list)
average_rmse = np.sqrt(average_squared_error)
correlation, _ = pearsonr(predictions, actual_values)

print(f"LOOCV - Average Root Mean Squared Error: {average_rmse}")
print(f"Pearson Correlation Coefficient: {correlation}")


IndexError: list index out of range

In [41]:
import statsmodels
def structural_coefficients(model: statsmodels.regression.linear_model.RegressionResultsWrapper, 
                            data: pd.DataFrame) -> pd.DataFrame:
    """
    This function calculates the structural coefficients of a linear regression model, along with their 
    associated model coefficients (beta weights). It also computes a 'suppressor index' which indicates 
    the likelihood of a variable being a suppressor variable (high beta weight, near-zero structural coefficient).
    A suppressor index over 10 is a good heuristic for identifying a suppressor variable
    
    Parameters:
    model (statsmodels.regression.linear_model.RegressionResultsWrapper): The fitted linear regression model.
    data (pd.DataFrame): The dataset used in the model.
    
    Returns:
    structural_coefs_df (pd.DataFrame): A dataframe containing the predictors, their structural coefficients,
                                        model coefficients (beta weights), and suppressor index.
                                        
                                        If the sum of the structure coefficients is higher than 1, they are correlated (multicollinear)
    """
    # Calculating the predicted values
    y_predicted = model.predict(data)

    # Creating a temporary dataframe to store interaction terms
    temp_df = pd.DataFrame()
    
    # Calculating the structural coefficients
    structural_coefs = {}
    for pred in model.params.index:
        if pred == 'Intercept':
            continue

        # Check if the predictor is an interaction term
        if ':' in pred:
            # Split the interaction term into its components
            components = pred.split(':')
            # Multiply the components and store the result in the temporary dataframe
            temp_product = data[components[0]]
            for component in components[1:]:
                temp_product *= data[component]
            temp_df[pred] = temp_product
        else:
            temp_df[pred] = data[pred]
        
        coef, _ = pearsonr(temp_df[pred], y_predicted)
        structural_coefs[pred] = np.square(coef)

    # Calculating the model coefficients
    model_coefs = model.params.drop('Intercept')

    # Creating a dataframe to store the results
    structural_coefs_df = pd.DataFrame(list(zip(structural_coefs.keys(), structural_coefs.values(), model_coefs.values)), 
                                       columns=['predictor', 'structural_coefficient', 'model_coefficient'])

    # Adding suppressor index column
    structural_coefs_df['suppressor_index'] = structural_coefs_df['model_coefficient'].abs() / structural_coefs_df['structural_coefficient']

    # Sorting the dataframe by the structural coefficients in descending order
    structural_coefs_df.sort_values(by='structural_coefficient', ascending=False, inplace=True)
    structural_coefs_df.reset_index(drop=True, inplace=True)

    return structural_coefs_df

In [ ]:
squared_structural_coefs_df = structural_coefficients(results, data_df.copy())
squared_structural_coefs_df.to_csv(os.path.join(out_dir, 'structural_coefficient_analysis.csv'))
print('saved to: ', os.path.join(out_dir, 'structural_coefficient_analysis.csv'))
display(squared_structural_coefs_df)

In [ ]:
from scipy import sqrt, mean

# Calculate the squared errors
squared_errors = (data_df['percent_change_adascog11'] - results.fittedvalues) ** 2

# Calculate the mean of the squared errors
mse = mean(squared_errors)

# Calculate the root mean squared error
rmse = sqrt(mse)

print("Root Mean Squared Error:", rmse)


In [ ]:
from calvin_utils.statistical_measurements import model_diagnostics
model_diagnostics(results)

Compare Two Lienar Regressions Using F-Test (ANOVA_LM In Statsmodels)

In [ ]:
from statsmodels.stats.api import anova_lm
smaller_formula = 'outcome ~ Subiculum_Connectivity'

larger_formula = 'outcome ~ Age*Subiculum_Connectivity'


#----------------------------------------------------------------DO NOT TOUCH!----------------------------------------------------------------
table1 = anova_lm(smf.ols(smaller_formula, data=data_df).fit(), smf.ols(larger_formula, data=data_df).fit())
print(table1)

In [ ]:
factor_groups = data_df.groupby(["Age", "Subiculum_Connectivity"])


#----------------------------------------------------------------do not touch
infl = results.get_influence()
resid = infl.resid_studentized_internal
plt.figure(figsize=(6, 6))
for values, group in factor_groups:
    i, j = values
    idx = group.index
    plt.scatter(
        data_df['Age'][idx],
        resid[idx],
        # marker=symbols[j],
        # color=colors[i - 1],
        s=144,
        edgecolors="black",
    )
plt.xlabel("X")
plt.ylabel("standardized resids")

## Use 2D Interaction Plot to Visualize Interactions


This code is designed to create an interaction plot to visualize the effects of two factors and their interaction on the outcome variable.

The interaction_plot function takes as input a dataframe, two factors (x_one and x_two), two corresponding labels for the conditions when the values of these factors are under the mean (x_one_under_mean and x_two_under_mean) and over the mean (x_one_over_mean and x_two_over_mean), and the response variable (outcome). If binarize is set to True, it converts the two factors into binary variables based on whether their values are above or below the mean. The function then creates a mapping for the x_two variable to numerical values for the purpose of plotting.

It uses the interaction_plot function from the statsmodels package to create the plot. In the plot, x_two is represented on the x-axis, x_one is used to color the lines, and the outcome variable is plotted on the y-axis. The function also sets the labels for the x and y axes and the tick labels on the x-axis according to the inputs provided.

The function also allows for saving the plot to an output directory specified by the user. If save is set to True, it saves the plot in both PNG and SVG formats.

In [ ]:
from statsmodels.graphics.api import interaction_plot

def two_dimensional_interaction_plot(data_df, 
                    x_one, x_one_under_mean, x_one_over_mean,
                    x_two, x_two_under_mean, x_two_over_mean, 
                    response, 
                    binarize=True,
                    x_label='Subiculum Connectivity', y_label='Percent Improvement (MDRS)',
                    save=False, out_dir=None):
    """
    Function to create an interaction plot.
    
    Parameters
    ----------
    data_df : pandas.DataFrame
        The dataframe containing the data.
    x_one, x_two : str
        Column names of the two factors.
    x_one_under_mean, x_two_under_mean : str
        Labels to be used when the values of x_one and x_two are under the mean, respectively.
    x_one_over_mean, x_two_over_mean : str
        Labels to be used when the values of x_one and x_two are over the mean, respectively.
    response : str
        Column name of the outcome variable.
    binarize : bool, optional
        Whether to convert x_one and x_two into binary variables.
    x_label, y_label : str, optional
        Labels for the x-axis and y-axis.
    save : bool, optional
        Whether to save the plot.
    out_dir : str, optional
        Directory where the plot will be saved.

    Returns
    -------
    pandas.DataFrame
        The original dataframe with modified x_one and x_two if binarize is True.
    """
    
    # Binarize x_two variable
    if binarize:
        data_df[x_two] = np.where(data_df[x_two] <= data_df[x_two].mean(), f'{x_two_under_mean}', f'{x_two_over_mean}')

    # Binarize x_one variable
    if binarize:
        data_df[x_one] = np.where(data_df[x_one] <= data_df[x_one].mean(), f'{x_one_under_mean}', f'{x_one_over_mean}')
    
    # Map the x_two categories to numbers for plotting
    mapping = {x_two_under_mean: 0, x_two_over_mean: 1}
    data_df[x_two] = data_df[x_two].map(mapping)
    
    # Create an interaction plot with the specified factors and response
    fig, ax = plt.subplots()
    interaction_plot(x=data_df[x_two].values, trace=data_df[x_one].values, response=data_df[response].values, colors=['Red', 'Blue'], xlabel=x_label, ylabel=y_label, ax=ax)

    # Set the labels for the x-axis and y-axis, and the tick labels on the x-axis
    ax.set_xticks([0, 1])
    ax.set_xticklabels([x_two_under_mean, x_two_over_mean])
    ax.set_ylabel(f'{y_label}')
    ax.set_xlabel(f'{x_label}')

    # Save the plot if save is True
    if save:
        if out_dir is not None:
            fig.savefig(out_dir + '2D_interaction.png')
            fig.savefig(out_dir + '2D_interaction.svg')
            print(f'saved to: {out_dir}')

    # Display the plot
    plt.show()

    return data_df


In [ ]:
two_dimensional_interaction_plot(data_df.copy(), 
                 x_one='Age', x_one_under_mean='Young', x_one_over_mean='Old', 
                 x_two='Subiculum_Connectivity', x_two_under_mean='Low Connectivity', x_two_over_mean='High Connectivity', 
                 response='outcome', 
                 x_label='Subiculum Connectivity', 
                 y_label='Percent Improvement (MDRS)',
                 save=True, out_dir='./')


## Use response plane to visualize interaction effect
- This models the marginal distribution of variables
- If the model has 2 predictors and 1 response, then choose option A
- If the model has more than 2 predictors, then choose option B. You will need to manually vary across the additional predictors

2 variable method

In [ ]:
def create_interaction_plot(data_df: pd.DataFrame,
                            x_one: str,
                            x_two: str,
                            outcome: str,
                            results: GLMResults,
                            num_slices: int = 100,
                            out_dir: str = './',
                            labels: dict = None) -> plt.Figure:
    """
    Creates a 3D plot visualizing the interaction of two predictor variables on the outcome.
    
    Parameters
    ----------
    data_df : pd.DataFrame
        Dataframe containing the predictor variables and outcome.
    x_one : str
        Column name of the first predictor variable in the dataframe.
    x_two : str
        Column name of the second predictor variable in the dataframe.
    outcome : str
        Column name of the outcome variable in the dataframe.
    results : GLMResults
        Fitted model used for making predictions.
    num_slices : int, optional
        Number of slices to create in the 3D grid, default is 100.
    out_dir : str, optional
        Directory to save the output PNG and SVG files, default is the current directory.
        
    Returns
    -------
    plt.Figure
        The created figure.
    """
    
    # Create grid of predictor variable values
    x1 = np.linspace(min(data_df[x_one]), max(data_df[x_one]), num_slices)
    x2 = np.linspace(min(data_df[x_two]), max(data_df[x_two]), num_slices)
    x1v, x2v = np.meshgrid(x1, x2)

    # Set plot style
    sns.set_style('white')
    sns.set_palette('Greys', 1, desat=1)

    # Flattening the matrices to create a DataFrame for prediction
    X_grid = pd.DataFrame({
        x_one: x1v.ravel(),
        x_two: x2v.ravel(),
    })

    # Generate response values for grid
    y_pred = results.predict(X_grid).values.reshape(num_slices, num_slices)

    # Create a new figure for plotting
    fig = plt.figure(figsize=(15, 15))
    ax = fig.add_subplot(111, projection='3d')

    # Plot the surface
    ax.plot_surface(x1v, x2v, y_pred, cmap='bwr', alpha=0.8)

    # Set the axes labels
    format_label = lambda x: ' '.join(word.capitalize() for word in x.split('_'))
    # Set the axes labels
    if labels is not None:
        ax.set_xlabel(labels.get('x', format_label(x_one)))
        ax.set_ylabel(labels.get('y', format_label(x_two)))
        ax.set_zlabel(labels.get('z', format_label(outcome)))
    else:
        ax.set_xlabel(format_label(x_one))
        ax.set_ylabel(format_label(x_two))
        ax.set_zlabel(format_label(outcome))


    # Save the plot as PNG and SVG files
    fig.savefig(out_dir + '2_variable.png', dpi=300)
    fig.savefig(out_dir + '2_variable.svg', format='svg')
    print('Saved to file', out_dir)

    return fig


In [ ]:
x_one = 'Subiculum_Connectivity'
x_two = 'Age'
outcome = 'outcome'
#----------------------------------------------------------------DO NOT TOUCH THIS!----------------------------------------------------------------
create_interaction_plot(data_df, 
                        x_one=x_one, 
                        x_two=x_two, 
                        outcome=outcome, 
                        results=results, 
                        out_dir=out_dir, 
                        labels={'x': 'Subiculum Connectivity', 'y': 'Age', 'z': 'Percent Improvement (MDRS)'});

3 Variable Method

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import imageio
import os
from statsmodels.genmod.generalized_linear_model import GLMResults
from typing import List


def create_interaction_gifs(data_df: pd.DataFrame, 
                            x_one: str, 
                            x_two: str, 
                            x_three: str, 
                            outcome: str, 
                            results: GLMResults, 
                            num_slices: int = 100,
                            gif_duration: float = 0.3) -> List[str]:
    """
    Creates gifs visualizing the interaction of predictor variables on the outcome.
    
    Parameters
    ----------
    data_df : pd.DataFrame
        Dataframe containing the predictor variables and outcome.
    x_one : str
        Column name of the first predictor variable in the dataframe.
    x_two : str
        Column name of the second predictor variable in the dataframe.
    x_three : str
        Column name of the third predictor variable in the dataframe.
    outcome : str
        Column name of the outcome variable in the dataframe.
    results : GLMResults
        Fitted model used for making predictions.
    num_slices : int, optional
        Number of slices to create in the gifs, default is 100.
    gif_duration : float, optional
        Duration of each frame in the gif in seconds, default is 0.3.
        
    Returns
    -------
    List[str]
        List of file paths to the created gif files.
    """

    # Check if x_three is binary
    unique_values = data_df[x_three].unique()
    is_binary = len(unique_values) == 2

    # Create grid of predictor variable values
    x1 = np.linspace(min(data_df[x_one]), max(data_df[x_one]), num_slices)
    x2 = np.linspace(min(data_df[x_two]), max(data_df[x_two]), num_slices)
    
    # Set plot style
    sns.set_style('white')
    sns.set_palette('Greys', 1, desat=1)

    gif_paths = []

    if is_binary:
        for x_three_value in unique_values:
            # Flattening the matrices to create a DataFrame for prediction
            X_grid = pd.DataFrame({
                x_one: np.tile(x1, len(x2)),
                x_two: np.repeat(x2, len(x1)),
                x_three: np.ones_like(np.tile(x1, len(x2))) * x_three_value
            })

            # Generate response values for grid
            y_pred = results.predict(X_grid).values.reshape(len(x2), len(x1))

            # Create a new figure for plotting
            fig = plt.figure(figsize=(10, 7))
            ax = fig.add_subplot(111, projection='3d')

            # Plot the surface
            ax.plot_surface(np.tile(x1, (len(x2), 1)), np.repeat(x2[:, np.newaxis], len(x1), axis=1), y_pred, cmap='bwr', alpha=0.8)

            # Set the axes labels
            format_label = lambda x: ' '.join(word.capitalize() for word in x.split('_'))
            ax.set_xlabel(format_label(x_one))
            ax.set_ylabel(format_label(x_two))
            ax.set_zlabel(format_label(outcome))

            # Set the title with the actual value of x_three
            title = f'{x_three} = {x_three_value}'
            ax.set_title(title)

            # Save the plot as an image file
            image_path = f'plot_{x_three}_{x_three_value}.png'
            plt.savefig(image_path, dpi=300)
            gif_paths.append(image_path)
            plt.close()
    else:
        # Flattening the matrices to create a DataFrame for prediction
        x3 = np.linspace(min(data_df[x_three]), max(data_df[x_three]), num_slices)
        x1v, x2v, x3v = np.meshgrid(x1, x2, x3)

        X_grid = pd.DataFrame({
            x_one: x1v.ravel(),
            x_two: x2v.ravel(),
            x_three: x3v.ravel(),
        })

        # Generate response values for grid
        y_pred = results.predict(X_grid).values.reshape(num_slices, num_slices, num_slices)

        # Collect the file paths of images
        image_paths = []

        # Iterating over x3 (third predictor variable) and saving each plot as an image
        for i in range(num_slices):
            x3_slice = np.linspace(min(data_df[x_three]), max(data_df[x_three]), num_slices)[i]

            # Create a new figure for plotting
            fig = plt.figure(figsize=(10, 7))
            ax = fig.add_subplot(111, projection='3d')

            # Plot the surface
            ax.plot_surface(x1v[:, :, i], x2v[:, :, i], y_pred[:, :, i], cmap='bwr', alpha=0.8)

            # Set the axes labels
            format_label = lambda x: ' '.join(word.capitalize() for word in x.split('_'))
            ax.set_xlabel(format_label(x_one))
            ax.set_ylabel(format_label(x_two))
            ax.set_zlabel(format_label(outcome))

            # Set the title with the actual value of x_three
            title = f'{x_three} = {x3_slice:.2f}'
            ax.set_title(title)

            # Save the plot as an image file
            image_path = f'plot_slice_{i+1}.png'
            plt.savefig(image_path, dpi=300)
            image_paths.append(image_path)
            plt.close()

        # Create a gif from the image files
        gif_path = f'plots.gif'
        images = [imageio.imread(image_path) for image_path in image_paths]
        imageio.mimsave(gif_path, images, duration=gif_duration)
        gif_paths.append(gif_path)

        # Remove the individual image files
        for image_path in image_paths:
            os.remove(image_path)

    return gif_paths


# Example usage
# gifs = create_interaction_gifs(data_df, 'x_one', 'x_two', 'x_three', 'outcome', results)
# print("GIFs created:", gifs)


In [ ]:
x_one = 'Subiculum_Connectivity'
x_two = 'Subiculum_Grey_Matter' 
x_three = 'Age'
outcome = 'cognitive_improvement'
#----------------------------------------------------------------DO NOT TOUCH----------------------------------------------------------------
gifs = create_interaction_gifs(data_df, 
                               x_one, 
                               x_two, 
                               x_three, 
                               outcome, 
                               results)
print("GIFs created:", gifs)

4 Variable Method
- 4th variable is expected to be 1-hot encoded here

In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import imageio
import os
from statsmodels.genmod.generalized_linear_model import GLMResults
from typing import List


def create_interaction_gifs(data_df: pd.DataFrame, 
                            x_one: str, 
                            x_two: str, 
                            x_three: str, 
                            x_four: str, 
                            x_four_dict: dict,
                            outcome: str, 
                            results: GLMResults, 
                            num_slices: int = 100,
                            gif_duration: float = 0.3) -> List[str]:
    """
    Creates gifs visualizing the interaction of predictor variables on the outcome,
    and the effect of a one-hot encoded variable (x_four).
    
    Parameters
    ----------
    data_df : pd.DataFrame
        Dataframe containing the predictor variables and outcome.
    x_one : str
        Column name of the first predictor variable in the dataframe.
    x_two : str
        Column name of the second predictor variable in the dataframe.
    x_three : str
        Column name of the third predictor variable in the dataframe.
    x_four : str
        Column name of the one-hot encoded predictor variable (values should be 0 or 1) in the dataframe.
    x_four_list : list
        List of the x_four variables corresponding to their one hot encoded outcomes.
    outcome : str
        Column name of the outcome variable in the dataframe.
    results : GLMResults
        Fitted model used for making predictions.
    num_slices : int, optional
        Number of slices to create in the gifs, default is 100.
    gif_duration : float, optional
        Duration of each frame in the gif in seconds, default is 0.3.
        
    Returns
    -------
    List[str]
        List of file paths to the created gif files.
    """

    # Set plot style
    sns.set_style('white')
    sns.set_palette('Greys', 1, desat=1)

    gif_paths = []

    # Iterate over x_four values 0 and 1
    for x_four_value in [0, 1]:
        distribution_df = data_df.copy()
        distribution_df = distribution_df[distribution_df['Disease'] == x_four_value]
        # Create grid of predictor variable values
        x1 = np.linspace(min(distribution_df[x_one]), max(distribution_df[x_one]), num_slices)
        x2 = np.linspace(min(distribution_df[x_two]), max(distribution_df[x_two]), num_slices)
        x3 = np.linspace(min(distribution_df[x_three]), max(distribution_df[x_three]), num_slices)
        x1v, x2v, x3v = np.meshgrid(x1, x2, x3)
        
        # Flattening the matrices to create a DataFrame for prediction
        X_grid = pd.DataFrame({
            x_one: x1v.ravel(),
            x_two: x2v.ravel(),
            x_three: x3v.ravel(),
            x_four: np.ones_like(x1v).ravel() * x_four_value
        })

        # Generate response values for grid
        y_pred = results.predict(X_grid).values.reshape(num_slices, num_slices, num_slices)

        # Collect the file paths of images
        image_paths = []

        # Iterating over x3 (third predictor variable) and saving each plot as an image
        for i in range(num_slices):
            x3_slice = np.linspace(min(data_df[x_three]), max(data_df[x_three]), num_slices)[i]
            
            # Create a new figure for plotting
            fig = plt.figure(figsize=(10, 7))
            ax = fig.add_subplot(111, projection='3d')

            # Plot the surface
            ax.plot_surface(x1v[:, :, i], x2v[:, :, i], y_pred[:, :, i], cmap='bwr', alpha=0.8)

            # Set the axes labels
            format_label = lambda x: ' '.join(word.capitalize() for word in x.split('_'))
            ax.set_xlabel(format_label(x_one))
            ax.set_ylabel(format_label(x_two))
            ax.set_zlabel(format_label(outcome))
            
            # Set the title with the actual value of x_three and x_four
            title = f'{x_four_dict[x_four_value]}, {x_three} = {x3_slice:.2f}'
            ax.set_title(title)

            # Save the plot as an image file
            image_path = f'plot_{x_four}_{x_four_dict[x_four_value]}_slice_{i+1}.png'
            plt.savefig(image_path, dpi=300)
            image_paths.append(image_path)
            plt.close()

        # Create a gif from the image files
        gif_path = f'{x_four}_{x_four_dict[x_four_value]}_plots.gif'
        images = [imageio.imread(image_path) for image_path in image_paths]
        imageio.mimsave(gif_path, images, duration=gif_duration)
        gif_paths.append(gif_path)

        # Remove the individual image files
        for image_path in image_paths:
            os.remove(image_path)

    return gif_paths


# Example usage
# gifs = create_interaction_gifs(data_df, 'Subiculum_Connectivity', 'Subiculum_Grey_Matter', 'Age', 'Disease', 'outcome', results)
# print("GIFs created:", gifs)

In [ ]:
data_df

In [9]:
x_one = 'Subiculum_Connectivity'
x_two = 'Subiculum_Grey_Matter' 
x_three = 'Age'
x_four = 'Disease'
x_four_dict = {1: "Alzheimer's", 0: "Parkinson's"}
outcome = 'cognitive_improvement'
#----------------------------------------------------------------DO NOT TOUCH----------------------------------------------------------------
gifs = create_interaction_gifs(data_df, 
                               x_one, 
                               x_two, 
                               x_three, 
                               x_four,
                               x_four_dict,
                               outcome, 
                               results)
print("GIFs created:", gifs)

/var/folders/82/946tskyj68b6htgvndtppmz80000gp/T/ipykernel_25492/3155527796.py:114: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(image_path) for image_path in image_paths]
/var/folders/82/946tskyj68b6htgvndtppmz80000gp/T/ipykernel_25492/3155527796.py:114: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images = [imageio.imread(image_path) for image_path in image_paths]


GIFs created: ["Disease_Parkinson's_plots.gif", "Disease_Alzheimer's_plots.gif"]


5 Variable Method
- This is identical to the 4 variable method, but takes the 5th variable 
and calculates is at -2 and +2 standard deviations, then plots a response plane for each.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import imageio
import os
from statsmodels.genmod.generalized_linear_model import GLMResults
from typing import List


def create_interaction_gifs_with_5_variables(data_df: pd.DataFrame, 
                                             x_one: str, 
                                             x_two: str, 
                                             x_three: str, 
                                             x_four: str, 
                                             x_four_dict: dict,
                                             x_five: str,
                                             x_five_factor_levels: int,
                                             outcome: str, 
                                             results: GLMResults, 
                                             num_slices: int = 100,
                                             gif_duration: float = 0.3) -> List[str]:
    
    # Create grid of predictor variable values
    x1 = np.linspace(min(data_df[x_one]), max(data_df[x_one]), num_slices)
    x2 = np.linspace(min(data_df[x_two]), max(data_df[x_two]), num_slices)
    x3 = np.linspace(min(data_df[x_three]), max(data_df[x_three]), num_slices)
    x1v, x2v, x3v = np.meshgrid(x1, x2, x3)

    # Set plot style
    sns.set_style('white')
    sns.set_palette('Greys', 1, desat=1)

    gif_paths = []

    # Iterate over x_four values 0 and 1
    for x_four_value in [0, 1]:
        
        # Collect the file paths of images
        image_paths = []

        # Iterating over x3 (third predictor variable) and saving each plot as an image
        for i in range(num_slices):
            x3_slice = np.linspace(min(data_df[x_three]), max(data_df[x_three]), num_slices)[i]
            
            # Create a new figure for plotting
            fig = plt.figure(figsize=(10, 7))
            ax = fig.add_subplot(111, projection='3d')

            # Choose to use x_five mean
            x_five_mean = data_df[x_five].mean()
            x_five_std = data_df[x_five].std()
            if x_five_factor_levels == 1:
                x_five_level = [x_five_mean]
            # Choose to use x_five +/- 2 standard deviations
            else:
                x_five_level = [x_five_mean-(2*x_five_std), x_five_mean+(2*x_five_std)]
            for level in range(0, len(x_five_level)):
                
                # Flattening the matrices to create a DataFrame for prediction
                X_grid = pd.DataFrame({
                    x_one: x1v.ravel(),
                    x_two: x2v.ravel(),
                    x_three: x3v.ravel(),
                    x_four: np.ones_like(x1v).ravel() * x_four_value,
                    x_five: np.ones_like(x1v).ravel() * x_five_level[level]
                })

                # Generate response values for grid
                y_pred = results.predict(X_grid).values.reshape(num_slices, num_slices, num_slices)
                
                # Plot the surface
                cmap = 'viridis' if x_five_level[level] == data_df[x_five].mean() else ('cool' if x_five_level[level] < 0 else 'magma')
                ax.plot_surface(x1v[:, :, i], x2v[:, :, i], y_pred[:, :, i], cmap=cmap, alpha=0.5)

            # Set the axes labels
            format_label = lambda x: ' '.join(word.capitalize() for word in x.split('_'))
            ax.set_xlabel(format_label(x_one))
            ax.set_ylabel(format_label(x_two))
            ax.set_zlabel(format_label(outcome))
            
            # Set the title with the actual value of x_three and x_four
            title = f'{x_four_dict[x_four_value]}, {x_three} = {x3_slice:.2f}'
            ax.set_title(title)

            # Save the plot as an image file
            image_path = f'plot_{x_four}_{x_four_dict[x_four_value]}_slice_{i+1}.png'
            plt.savefig(image_path, dpi=300)
            image_paths.append(image_path)
            plt.close()

        # Create a gif from the image files
        gif_path = f'{x_four}_{x_four_dict[x_four_value]}_plots.gif'
        images = [imageio.imread(image_path) for image_path in image_paths]
        imageio.mimsave(gif_path, images, duration=gif_duration)
        gif_paths.append(gif_path)

        # Remove the individual image files
        for image_path in image_paths:
            os.remove(image_path)

    return gif_paths




In [ ]:
x_one = 'Subiculum_Connectivity'
x_two = 'Subiculum_Grey_Matter' 
x_three = 'Age'
x_four = 'Disease'
x_four_dict = {1: "Alzheimer's", 0: "Parkinson's"}
x_five = 'Subiculum_CSF'
x_five_factor_levels = 1
outcome = 'cognitive_improvement'
#----------------------------------------------------------------DO NOT TOUCH----------------------------------------------------------------
gifs = create_interaction_gifs_with_5_variables(data_df, 
                               x_one, 
                               x_two, 
                               x_three, 
                               x_four,
                               x_four_dict,
                               x_five,
                               x_five_factor_levels,
                               outcome, 
                               results)
print("GIFs created:", gifs)

## Comparison Within Joint Distribution

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import scipy.stats as stats
from tqdm import tqdm

def compute_statistics_and_create_dataframe(resampled_metrics, observed_metric, test_type, test_tails='two_tailed'):
    """
    Compute p-value and confidence interval and organize them in a DataFrame.
    
    :param resampled_metrics: List of metrics computed from the resampled or permuted data
    :param observed_metric: Observed metric computed from the original data
    :param comparison_method: Method for comparing responses ('correlation' or 'point_testing')
    :param test_type: Type of test performed ('resampling' or 'permutation')
    :param test_tails: Type of tail for p-value testing ('two_tailed', 'right_tailed', or 'left_tailed')
    :return: DataFrame with p-value and confidence interval
    """
    # Calculate p-value and confidence interval for low and high values
    if test_type =='permutation':
        if test_tails == 'two_tailed':
            print('Two-tailed p-value testing')
            p_value_low = np.mean(np.abs([x[0] for x in resampled_metrics]) >= np.abs(observed_metric[0]))
            p_value_high = np.mean(np.abs([x[1] for x in resampled_metrics]) >= np.abs(observed_metric[1]))
        elif test_tails == 'right_tailed':
            print('Right-tailed p-value testing')
            p_value_low = np.mean([x[0] for x in resampled_metrics] >= observed_metric[0])
            p_value_high = np.mean([x[1] for x in resampled_metrics] >= observed_metric[1])
        elif test_tails == 'left_tailed':
            print('Left-tailed p-value testing.')
            p_value_low = np.mean([x[0] for x in resampled_metrics] <= observed_metric[0])
            p_value_high = np.mean([x[1] for x in resampled_metrics] <= observed_metric[1])
        else:
            raise ValueError("Invalid value for test_tails. Must be one of 'two_tailed', 'right_tailed', or 'left_tailed'.")
    elif test_type == 'resampling':
        p_value_low = np.mean(np.sign(np.mean([x[0] for x in resampled_metrics])) != np.sign(observed_metric[0]))
        p_value_high = np.mean(np.sign(np.mean([x[1] for x in resampled_metrics])) != np.sign(observed_metric[1]))
    else:
        raise ValueError(f"Test type is not supported: {test_type}")
    
    conf_int_low = np.percentile([x[0] for x in resampled_metrics], [2.5, 97.5])
    conf_int_high = np.percentile([x[1] for x in resampled_metrics], [2.5, 97.5])

    # Create DataFrame
    data = {
        'Observed Metric': [observed_metric[0], observed_metric[1]],
        'P-value': [p_value_low, p_value_high],
        '95% CI Lower': [conf_int_low[0], conf_int_high[0]],
        '95% CI Upper': [conf_int_low[1], conf_int_high[1]]
    }
    index = ['Delta Y pred indep_var_2 +/- stdev holding indep_var_1 at -2stdev', 
                'Delta Y pred indep_var_2 +/- stdev holding indep_var_1 at +2stdev']

    return pd.DataFrame(data, index=[index])


def compare_responses(df, dependent_var, independent_var1, independent_var2, interaction=True):
    """
    Compare responses of a dataset using point testing method.
    
    :param df: DataFrame containing the dataset
    :param dependent_var: Name of the dependent variable
    :param independent_var1: Name of the first independent variable
    :param independent_var2: Name of the second independent variable
    :param interaction: Boolean indicating if interaction between independent variables should be considered
    :return: Comparison metric (absolute differences between low and high points)
    """
    if interaction:
        formula = f'{dependent_var} ~ {independent_var1} * {independent_var2}'
    else:
        formula = f'{dependent_var} ~ {independent_var1} + {independent_var2}'

    # Fit the model
    model = smf.ols(formula=formula, data=df).fit()

    # Evaluate at specific points
    diff_low, diff_high = evaluate_at_specific_points(df, independent_var1, independent_var2, model)

    # Return the absolute differences
    return abs(diff_low), abs(diff_high)

def evaluate_at_specific_points(df, independent_var1, independent_var2, model):
    """
    Evaluate the model at specific points.
    
    :param df: DataFrame containing the dataset
    :param independent_var1: Name of the first independent variable
    :param independent_var2: Name of the second independent variable
    :param model: Fitted model
    :return: Differences between low and high points
    """
    # Determine the low and high values of the independent variables
    iv1_sd = data_df[independent_var1].std()
    iv2_sd = data_df[independent_var2].std()
    iv1_mean = data_df[independent_var1].mean()
    iv2_mean = data_df[independent_var2].mean()

    # Create the dataframes for low_1 and high_1 predictions
    low_1_df = pd.DataFrame({
        independent_var1: iv1_mean - 2 * iv1_sd,
        independent_var2: [iv2_mean - 2 * iv2_sd, iv2_mean + 2 * iv2_sd]
    })

    high_1_df = pd.DataFrame({
        independent_var1: iv1_mean + 2 * iv1_sd,
        independent_var2: [iv2_mean - 2 * iv2_sd, iv2_mean + 2 * iv2_sd]
    })

    # Perform the predictions on the dataframes
    low_1_predictions = model.predict(low_1_df)
    high_1_predictions = model.predict(high_1_df)

    # Calculate the differences between the predictions
    difference_low = low_1_predictions[1] - low_1_predictions[0]
    difference_high = high_1_predictions[1] - high_1_predictions[0]

    return difference_low, difference_high

def resample_or_permutation_test(df, dependent_var, independent_var1, independent_var2, interaction=True, num_resamples=1000, test_type='permutation'):
    """
    Perform resampling or permutation test to compare responses of two datasets and return results in a DataFrame.
    
    :param df: DataFrame containing the dataset
    :param dependent_var: Name of the dependent variable
    :param independent_var1: Name of the first independent variable
    :param independent_var2: Name of the second independent variable
    :param interaction: Boolean indicating if interaction between independent variables should be considered
    :param comparison_method: Method for comparing responses ('correlation' or 'point_testing')
    :param num_slices: Number of values to take for each variable to form the grid (used in correlation)
    :param num_resamples: Number of resamples to perform
    :param test_type: Type of test to perform ('resampling' or 'permutation')
    :return: p-value and confidence interval
    """
    # Calculate observed metric
    observed_metric = compare_responses(df, dependent_var, independent_var1, independent_var2, interaction)
    
    # Initialize results
    resampled_metrics = []

    for i in tqdm(range(num_resamples)):
        # Perform resampling or permutation
        if test_type == 'resampling':
            sample_df = df.sample(frac=1, replace=True)
        elif test_type == 'permutation':
            sample_df = df.copy()
            sample_df[dependent_var] = np.random.permutation(sample_df[dependent_var].values)
        else:
            raise ValueError("test_type must be either 'resampling' or 'permutation'.")
        
        # Calculate resampled metric
        resampled_metric = compare_responses(sample_df, dependent_var, independent_var1, independent_var2, interaction)
        resampled_metrics.append(resampled_metric)
    
    # Calculate statistics and create DataFrame
    results_dataframe = compute_statistics_and_create_dataframe(resampled_metrics, observed_metric, test_type)
    
    return results_dataframe


In [ ]:
data_df

In [ ]:
#Define dataframe 1 and dataframe 2 for comaprison by permutation test
df_1 = data_df[data_df['Disease'] == 1]
df_2 = data_df[data_df['Disease'] == 0]
dependent_var = 'outcome'
independent_var1 = 'Age'
independent_var2 = 'Subiculum_Connectivity'
enable_interaction = True
num_iterations = 10000
#----------------------------------------------------------------DO NOT TOUCH----------------------------------------------------------------
permutation_results = resample_or_permutation_test(df = df_2,
                                                    dependent_var = dependent_var,
                                                    independent_var1 = independent_var1,
                                                    independent_var2 = independent_var2,
                                                    interaction = enable_interaction,
                                                    num_resamples=num_iterations, 
                                                    test_type='permutation')
permutation_results

In [ ]:
#----------------------------------------------------------------DO NOT TOUCH----------------------------------------------------------------
bootstrap_results = resample_or_permutation_test(df = df_2,
                                                    dependent_var = dependent_var,
                                                    independent_var1 = independent_var1,
                                                    independent_var2 = independent_var2,
                                                    interaction = enable_interaction,
                                                    num_resamples=num_iterations, 
                                                    test_type='resampling')
bootstrap_results

## Comparison Between Joint Distributions

**Comparison of 2 Joint Distributions**
1) Joint Distributions and Multiple Regression:

The concept of "joint distributions" refers to the combined distribution of multiple variables, taking into account their mutual relationships. In the context of multiple regression analysis, we utilize a statistical model to examine the joint distribution of the independent variables and their relationship with the dependent variable. By fitting a regression model, such as the one used in this code, we can investigate how the independent variables collectively influence the dependent variable.

In the provided code, we compare the responses of two datasets based on their joint distributions. This means that we analyze the relationship between the independent variables (independent_var1 and independent_var2) as a pair and their collective influence on the dependent variable. We fit separate regression models for each dataset, considering both independent variables as predictors and the dependent variable as the response.

Analyzing the joint distributions allows us to gain insights into how the independent variables interact with each other and jointly impact the dependent variable. By considering the joint relationship between the variables, we can understand how changes in one independent variable may affect the response when the other independent variable is also taken into account.

2) Permutation vs Bootstrap:

The code employs two different approaches for hypothesis testing: permutation testing and bootstrap resampling. Both methods aim to assess the significance of the observed metrics by generating alternative datasets.

Permutation testing involves randomly permuting the values of the dependent variable while keeping the independent variables unchanged. This process breaks the relationship between the dependent and independent variables, creating new datasets under the assumption of no association. By comparing the observed metric with metrics obtained from the permuted datasets, we can estimate p-values and confidence intervals, providing a measure of the statistical significance of our results. A permuted dataset is one wherein the independent variables are unrelated to the dependent variable. They are the null distribution. 

Bootstrap resampling, on the other hand, involves randomly sampling data points from the original datasets with replacement. This resampling process allows us to create multiple datasets of the same size as the original datasets. By generating new datasets through resampling, we can examine the distribution of the metrics of interest and estimate their variability. This information can be used to estimate confidence intervals and assess the statistical significance of our findings. A bootstrap resampling process creates alternative datasets wherein the independent variables are still related to the dependent variable. These define significance by demonstrating the confidence intervals of the dataset do not cross a threshold, which is typically 0.

3) Correlation vs Point-test:

The code provides two different methods for comparing responses based on the joint distributions: correlation and point testing.

The correlation analysis aims to assess the spatial correlation between the joint distributions of the two datasets. This analysis examines the overall patterns and similarities in the responses across the range of the joint distributions. By calculating the correlation coefficient between the predicted responses, we can determine if there is a consistent relationship between the joint distributions of the independent variables and the dependent variable across the datasets. Then, we compare this to bootstrapped or permuted datasets to derive significance estimates.

In contrast, point testing focuses on evaluating the differences in the predicted responses at specific points of the joint distributions. Specifically, we examine the differences at +/- 2 standard deviations of the independent variables. This analysis allows us to determine if the differences between the responses at these specific points significantly vary between the two datasets. By comparing the differences in response values at these critical points, we can identify regions of the joint distributions where the relationship between the independent variables and the dependent variable differs significantly. The purpose of this is to condense a 3-dimensional joint distribution into 2-dimensions at set values which is good for generating figures. Then, we compare this to bootstrapped or permuted datasets to derive significance estimates.

In summary, comparing the joint distributions enables us to analyze the combined effects of the independent variables on the dependent variable. The correlation and point testing methods offer different perspectives on the relationship between the variables, while permutation testing and bootstrap resampling provide means to assess the statistical significance of our findings.

**Instructions**
Testing: 'correlation' or 'point_testing'
Significance Testing: 'permutation' or 'resampling'

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import scipy.stats as stats
from tqdm import tqdm

def compute_statistics_and_create_dataframe(resampled_metrics, observed_metric, comparison_method, test_type, test_tails='two_tailed'):
    """
    Compute p-value and confidence interval and organize them in a DataFrame.
    
    :param resampled_metrics: List of metrics computed from the resampled or permuted data
    :param observed_metric: Observed metric computed from the original data
    :param comparison_method: Method for comparing responses ('correlation' or 'point_testing')
    :return: DataFrame with p-value and confidence interval
    """
    if comparison_method == 'correlation':
        # Calculate p-value and confidence interval
        p_value = np.mean(np.abs(resampled_metrics) >= np.abs(observed_metric))
        conf_int = np.percentile(resampled_metrics, [2.5, 97.5])

        # Create DataFrame
        data = {
            'Observed Metric': [observed_metric],
            'P-value': [p_value],
            '95% CI Lower': [conf_int[0]],
            '95% CI Upper': [conf_int[1]]
        }
        index = 'Spatial correlation of the joint distributions'
    
    elif comparison_method == 'point_testing':
        # Calculate p-value and confidence interval for low and high values
        if test_type =='permutation':
            if test_tails == 'two_tailed':
                print('Two-tailed p-value testing')
                p_value_low = np.mean(np.abs([x[0] for x in resampled_metrics]) >= np.abs(observed_metric[0]))
                p_value_high = np.mean(np.abs([x[1] for x in resampled_metrics]) >= np.abs(observed_metric[1]))
            elif test_tails == 'right_tailed':
                print('Right-tailed p-value testing')
                p_value_low = np.mean([x[0] for x in resampled_metrics] >= observed_metric[0])
                p_value_high = np.mean([x[1] for x in resampled_metrics] >= observed_metric[1])
            elif test_tails == 'left_tailed':
                print('Left-tailed p-value testing.')
                p_value_low = np.mean([x[0] for x in resampled_metrics] <= observed_metric[0])
                p_value_high = np.mean([x[1] for x in resampled_metrics] <= observed_metric[1])
            else:
                raise ValueError("Invalid value for test_tails. Must be one of 'two_tailed', 'right_tailed', or 'left_tailed'.")
        elif test_type == 'resampling':
            p_value_low = np.mean(np.sign(np.mean([x[0] for x in resampled_metrics])) != (np.sign(np.mean([x[0] for x in resampled_metrics]))))
            p_value_high = np.mean(np.sign(np.mean([x[1] for x in resampled_metrics])) != (np.sign(np.mean([x[1] for x in resampled_metrics]))))
        else:
            raise ValueError (f'Test type is not supported {test_type}')
        conf_int_low = np.percentile(resampled_metrics[0], [2.5, 97.5])
        conf_int_high = np.percentile(resampled_metrics[1], [2.5, 97.5])

        # Create DataFrame
        data = {
            'Observed Metric': [observed_metric[0], observed_metric[1]],
            'P-value': [p_value_low, p_value_high],
            '95% CI Lower': [conf_int_low[0], conf_int_high[0]],
            '95% CI Upper': [conf_int_low[1], conf_int_high[1]]
        }
        index = ['Delta Y pred indep_var_2 +/- stdev holding indep_var_1 at -2stdev', 
                 'Delta Y pred indep_var_2 +/- stdev holding indep_var_1 at +2stdev']

    else:
        raise ValueError("comparison_method must be either 'correlation' or 'point_testing'.")

    return pd.DataFrame(data, index=[index])


def calculate_predicted_response(data_df, independent_var1, independent_var2, model, num_slices=100):
    """
    Calculate predicted response values for a grid of values of two independent variables.
    
    :param data_df: DataFrame containing data
    :param independent_var1: Name of the first independent variable
    :param independent_var2: Name of the second independent variable
    :param model: Fitted model
    :param num_slices: Number of values to take for each variable to form the grid
    :return: Flattened array of predicted responses
    """
    x1 = np.linspace(min(data_df[independent_var1]), max(data_df[independent_var1]), num_slices)
    x2 = np.linspace(min(data_df[independent_var2]), max(data_df[independent_var2]), num_slices)
    x1v, x2v = np.meshgrid(x1, x2)

    X_grid = pd.DataFrame({
        independent_var1: x1v.ravel(),
        independent_var2: x2v.ravel(),
    })

    y_pred = model.predict(X_grid).values.reshape(num_slices, num_slices)

    return y_pred.ravel()

def evaluate_at_specific_points(data_df, independent_var1, independent_var2, model):
    """
    Evaluate the model at specific points: +/- 2 standard deviations of the independent variables.

    :param data_df: DataFrame containing data
    :param independent_var1: Name of the first independent variable
    :param independent_var2: Name of the second independent variable
    :param model: Fitted model
    :return: Differences between the predictions at specific points
    """
    iv1_sd = data_df[independent_var1].std()
    iv2_sd = data_df[independent_var2].std()
    iv1_mean = data_df[independent_var1].mean()
    iv2_mean = data_df[independent_var2].mean()

    # Create the dataframes for low_1 and high_1 predictions
    low_1_df = pd.DataFrame({
        independent_var1: iv1_mean - 2 * iv1_sd,
        independent_var2: [iv2_mean - 2 * iv2_sd, iv2_mean + 2 * iv2_sd]
    })

    high_1_df = pd.DataFrame({
        independent_var1: iv1_mean + 2 * iv1_sd,
        independent_var2: [iv2_mean - 2 * iv2_sd, iv2_mean + 2 * iv2_sd]
    })

    # Perform the predictions on the dataframes
    low_1_predictions = model.predict(low_1_df)
    high_1_predictions = model.predict(high_1_df)

    # Calculate the differences between the predictions
    difference_low = low_1_predictions[1] - low_1_predictions[0]
    difference_high = high_1_predictions[1] - high_1_predictions[0]

    return difference_low, difference_high

def compare_responses(df1, df2, dependent_var, independent_var1, independent_var2, interaction=True, comparison_method='correlation', num_slices=100):
    """
    Compare responses of two datasets using correlation or point testing method.
    
    :param df1: DataFrame containing first dataset
    :param df2: DataFrame containing second dataset
    :param dependent_var: Name of the dependent variable
    :param independent_var1: Name of the first independent variable
    :param independent_var2: Name of the second independent variable
    :param interaction: Boolean indicating if interaction between independent variables should be considered
    :param comparison_method: Method for comparing responses ('correlation' or 'point_testing')
    :param num_slices: Number of values to take for each variable to form the grid (used in correlation)
    :return: Comparison metric
    """
    if interaction:
        formula = f'{dependent_var} ~ {independent_var1} * {independent_var2}'
    else:
        formula = f'{dependent_var} ~ {independent_var1} + {independent_var2}'

    # Fit the models
    model1 = smf.ols(formula=formula, data=df1).fit()
    model2 = smf.ols(formula=formula, data=df2).fit()

    if comparison_method == 'correlation':
        # Calculate predicted responses
        response_1 = calculate_predicted_response(df1, independent_var1, independent_var2, model1, num_slices)
        response_2 = calculate_predicted_response(df2, independent_var1, independent_var2, model2, num_slices)

        # Calculate correlation between the predicted responses
        return stats.pearsonr(response_1, response_2)[0]

    elif comparison_method == 'point_testing':
        # Evaluate at specific points
        diff_low_1, diff_high_1 = evaluate_at_specific_points(df1, independent_var1, independent_var2, model1)
        diff_low_2, diff_high_2 = evaluate_at_specific_points(df2, independent_var1, independent_var2, model2)
        
        # Return the absolute differences
        return abs(diff_low_1 - diff_low_2), abs(diff_high_1 - diff_high_2)

    else:
        raise ValueError("comparison_method must be either 'correlation' or 'point_testing'.")

# Update the resample_or_permutation_test function
def resample_or_permutation_test(df1, df2, dependent_var, independent_var1, independent_var2, interaction=True, comparison_method='correlation', num_slices=100, num_resamples=1000, test_type='permutation'):
    """
    Perform resampling or permutation test to compare responses of two datasets and return results in a DataFrame.
    
    :param df1: DataFrame containing first dataset
    :param df2: DataFrame containing second dataset
    :param dependent_var: Name of the dependent variable
    :param independent_var1: Name of the first independent variable
    :param independent_var2: Name of the second independent variable
    :param interaction: Boolean indicating if interaction between independent variables should be considered
    :param comparison_method: Method for comparing responses ('correlation' or 'point_testing')
    :param num_slices: Number of values to take for each variable to form the grid (used in correlation)
    :param num_resamples: Number of resamples to perform
    :param test_type: Type of test to perform ('resampling' or 'permutation')
    :return: p-value and confidence interval
    """
    # Calculate observed metric
    observed_metric = compare_responses(df1, df2, dependent_var, independent_var1, independent_var2, interaction, comparison_method, num_slices)
    
    # Initialize results
    resampled_metrics = []

    for i in tqdm(range(num_resamples)):
        # Perform resampling or permutation
        if test_type == 'resampling':
            sample_df1 = df1.sample(frac=1, replace=True)
            sample_df2 = df2.sample(frac=1, replace=True)
        elif test_type == 'permutation':
            sample_df1 = df1.copy()
            sample_df2 = df2.copy()
            sample_df1[dependent_var] = np.random.permutation(sample_df1[dependent_var].values)
            sample_df2[dependent_var] = np.random.permutation(sample_df2[dependent_var].values)
        else:
            raise ValueError("test_type must be either 'resampling' or 'permutation'.")
        
        # Calculate resampled metric
        resampled_metric = compare_responses(sample_df1, sample_df2, dependent_var, independent_var1, independent_var2, interaction, comparison_method, num_slices)
        resampled_metrics.append(resampled_metric)
    
    # Calculate statistics and create DataFrame
    results_dataframe = compute_statistics_and_create_dataframe(resampled_metrics, observed_metric, comparison_method, test_type)
    
    return results_dataframe


In [ ]:
#Define dataframe 1 and dataframe 2 for comaprison by permutation test
df_1 = data_df[data_df['Disease'] == 1]
df_2 = data_df[data_df['Disease'] == 0]
dependent_var = 'outcome'
independent_var1 = 'Age'
independent_var2 = 'Subiculum_Connectivity'
enable_interaction = True
num_iterations = 10000
comparison_method = 'correlation' 
#----------------------------------------------------------------DO NOT TOUCH----------------------------------------------------------------
permutation_results = resample_or_permutation_test(df1 = df_1,
                                                    df2 = df_2,
                                                    dependent_var = dependent_var,
                                                    independent_var1 = independent_var1,
                                                    independent_var2 = independent_var2,
                                                    interaction = enable_interaction,
                                                    test_type = 'permutation',
                                                    comparison_method = comparison_method,
                                                    num_resamples = num_iterations)
permutation_results

In [ ]:
#----------------------------------------------------------------DO NOT TOUCH----------------------------------------------------------------
bootstrap_results = resample_or_permutation_test(df1 = df_1,
                                                    df2 = df_2,
                                                    dependent_var = dependent_var,
                                                    independent_var1 = independent_var1,
                                                    independent_var2 = independent_var2,
                                                    interaction = enable_interaction,
                                                    test_type = 'resampling',
                                                    comparison_method = comparison_method,
                                                    num_resamples = num_iterations)
bootstrap_results

## Generate a Fancy Response Tensor

In [ ]:
from calvin_utils.import_matrices import import_matrices_from_folder
from calvin_utils.generate_nifti import nifti_from_matrix
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers

#Name variables to plot
var_one = 'Limbic'
var_two = 'Ventral_Attention'
val_var_two = 65.4

#get conectivity values of interest
matrix_path = '/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/seed_generated_networks/yeo_networks_from_thick_yeo_seeds/all_05_limbic_T.nii'
matrix = import_matrices_from_folder(matrix_path, file_pattern='')

#get conectivity values of interest
matrix_path_2 = '/Users/cu135/Dropbox (Partners HealthCare)/memory/functional_networks/seed_generated_networks/yeo_networks_from_thick_yeo_seeds/all_04_ventral_attention_T.nii'
matrix_2 = import_matrices_from_folder(matrix_path, file_pattern='')


#------------------------------------------------------------------------------------------------DO NOT MODIFY!------------------------------------------------------------------------------------------------
#Set reression input values into a dataframe
input_df = pd.DataFrame()
input_df[var_one] = matrix.iloc[:,0]
input_df[var_two] = matrix_2.iloc[:,0]

from nimlab import datasets as nimds
mni_mask = nimds.get_img("mni_icbm152")
mask_data = mni_mask.get_fdata().flatten()
brain_indices = np.where(mask_data > 0)[0]
input_df = input_df.iloc[brain_indices, :]

#Standardize matrix via z score
from calvin_utils.z_score_matrix import z_score_matrix
for col in input_df.columns:
    input_df.loc[:, col] = z_score_matrix(input_df.loc[:, col])

#Display results
display(input_df)


In [ ]:
import tqdm
#Generate a new matrix from the input dataframe
responses = []

#Work on a voxel-wise basis by iterating over index
for i in tqdm.tqdm(range(0, len(input_df))):
    #Do not calculate on zero values as intercept will be applied in the regressoin
    if np.sum(input_df.iloc[i,0]) != 0:
        #Assign a temporary dataframe with values that the statsmodels model is expecting
        temp_df = pd.DataFrame({var_one: input_df.iloc[i,0], var_two: input_df.iloc[i,1]}, index=['temp_vals'])
        #Calculate the voxelwise predicted outcome at a given voxel
        responses.append(results.predict(temp_df)[0])
    else:
        #If voxel is zero-connectivity, assign zero so as to avoid application of intercept
        responses.append(0)
        
#Store responses in a dataframe
response_df = pd.DataFrame()
response_df['response_topology'] = responses
display(response_df)

In [ ]:

#Place back in brain mask
mask_data[brain_indices] = response_df.loc[:, 'response_topology']
response_toplogy = mask_data

In [ ]:
from calvin_utils.generate_nifti import view_and_save_nifti
matrix_img = view_and_save_nifti(response_toplogy, out_dir)
matrix_img

## Data Driven Search for Most Relevant Regressors
Combines:
- All possible subsets regression combined with leave-one-out cross validation
- Identifies formula with lowest RMSE and highest Pearson Correlation of predictor to observed outcome values
- Presents the optimal formulas and allows selection of which formula to interpret with structural analysis and coefficient analysis

In [ ]:
import pandas as pd
import numpy as np
import itertools
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from tqdm import tqdm

def generate_interaction_terms(df, predictors, max_interaction_level=3):
    interaction_df = df[predictors].copy()

    def add_interaction_terms(df, columns, interaction_level):
        if interaction_level == 1:
            return df

        for col_combination in itertools.combinations(columns, interaction_level):
            interaction_term = ':'.join(col_combination)
            interaction_values = np.prod(df[list(col_combination)], axis=1)
            df[interaction_term] = interaction_values

        return add_interaction_terms(df, columns, interaction_level - 1)

    return add_interaction_terms(interaction_df, predictors, max_interaction_level)

def loocv_regression(df, interaction_df, formula, outcome_var):
    full_df = interaction_df.copy()
    full_df[outcome_var] = df[outcome_var]
    model = sm.formula.ols(formula, data=full_df).fit()
    y_actual, y_predicted = [], []
    
    for i in range(len(df)):
        x_test = interaction_df.iloc[[i]]
        x_train = interaction_df.drop(i)
        y_train = df[outcome_var].drop(i)
        
        model_train_df = x_train.join(y_train)
        model_train = sm.formula.ols(formula, data=model_train_df).fit()
        y_actual.append(df[outcome_var].iloc[i])
        y_predicted.append(model_train.predict(x_test).values[0])
    
    rmse = np.sqrt(mean_squared_error(y_actual, y_predicted))
    corr, _ = pearsonr(y_actual, y_predicted)
    return rmse, corr

def all_subsets_regression(df, outcome_var):
    predictor_columns = [col for col in df.columns if col != outcome_var]
    interaction_df = generate_interaction_terms(df, predictor_columns, max_interaction_level=3)

    results = []
    for n_predictors in tqdm(range(1, len(predictor_columns) + 1)):
        for predictor_combination in itertools.combinations(predictor_columns, n_predictors):
            formula = f"{outcome_var} ~ {' + '.join(predictor_combination)}"
            loocv_rmse, loocv_corr = loocv_regression(df, interaction_df, formula, outcome_var)

            results.append({
                'formula': formula,
                'rmse': loocv_rmse,
                'correlation': loocv_corr
            })

    results_df = pd.DataFrame(results)
    results_df.sort_values(by='correlation', ascending=False, inplace=True)
    results_df.reset_index(drop=True, inplace=True)
    return results_df


def all_subsets_regression_no_loocv(df, outcome_var):
    predictor_columns = [col for col in df.columns if col != outcome_var]

    results = []
    for n_predictors in tqdm(range(1, len(predictor_columns) + 1)):
        for predictor_combination in itertools.combinations(predictor_columns, n_predictors):
            formula = f"{outcome_var} ~ {' + '.join(predictor_combination)}"
            model = sm.formula.ols(formula, data=df).fit()

            y_actual = df[outcome_var].values
            y_predicted = model.predict(df[predictor_columns])

            rmse = np.sqrt(mean_squared_error(y_actual, y_predicted))
            corr, _ = pearsonr(y_actual, y_predicted)

            results.append({
                'formula': formula,
                'rmse': rmse,
                'correlation': corr
            })

    results_df = pd.DataFrame(results)
    results_df.sort_values(by='correlation', ascending=False, inplace=True)
    results_df.reset_index(drop=True, inplace=True)
    return results_df

In [ ]:
import pandas as pd
import numpy as np
import itertools
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from tqdm import tqdm


def all_subsets_regression(df, outcome_var, limit_to_two_way_interactions=False):
    predictor_columns = [col for col in df.columns if col != outcome_var]

    results = []
    for n_predictors in tqdm(range(1, len(predictor_columns) + 1)):
        for predictor_combination in itertools.combinations(predictor_columns, n_predictors):
            # Check whether to limit interactions to two-way
            if limit_to_two_way_interactions and n_predictors > 1:
                # Generate combinations for two-way interactions
                interaction_combinations = itertools.combinations(predictor_combination, 2)
                interaction_terms = [' : '.join(interaction) for interaction in interaction_combinations]
                # Include both individual predictors and interaction terms
                formula = f"{outcome_var} ~ {' + '.join(predictor_combination)} + {' + '.join(interaction_terms)}"
            else:
                # Original formula generation with all possible interactions
                formula = f"{outcome_var} ~ {' * '.join(predictor_combination)}"
            
            print(formula)
            loocv_rmse, loocv_corr = loocv_regression(df, formula, outcome_var)

            results.append({
                'formula': formula,
                'rmse': loocv_rmse,
                'correlation': loocv_corr
            })

    results_df = pd.DataFrame(results)
    results_df.sort_values(by='correlation', ascending=False, inplace=True)
    results_df.reset_index(drop=True, inplace=True)
    return results_df


def loocv_regression(df, formula, outcome_var):
    model = sm.formula.ols(formula, data=df).fit()
    y_actual, y_predicted = [], []

    for i in range(len(df)):
        index_label = df.index[i]
        x_test = df.iloc[[i]].drop(outcome_var, axis=1)
        x_train = df.drop(index_label).drop(outcome_var, axis=1)
        y_train = df[outcome_var].drop(index_label)

        model_train_df = x_train.join(y_train)
        model_train = sm.formula.ols(formula, data=model_train_df).fit()
        y_actual.append(df[outcome_var].iloc[i])
        y_predicted.append(model_train.predict(x_test).values[0])

    rmse = np.sqrt(mean_squared_error(y_actual, y_predicted))
    corr, _ = pearsonr(y_actual, y_predicted)
    return rmse, corr


# Example usage:
# df = pd.read_csv('your_data.csv')
# outcome_var = 'your_outcome_variable'
# results_df = all_subsets_regression(df, outcome_var)
# print(results_df.head())

In [ ]:
outcome_var = 'outcome'
limit_to_two_way_interactions = False
#----------------------------------------------------------------DO NOT TOUCH----------------------------------------------------------------
df = data_df.copy()
results_df = all_subsets_regression(df, outcome_var, limit_to_two_way_interactions=limit_to_two_way_interactions)
display(results_df)

In [ ]:
print('Top 20 formulas by Correlation of y-hat to y')
pd.set_option('display.max_colwidth', None)
results_df.head(20)

In [ ]:
print('Top 20 formulas by RMSE of y-hat to y')
# Sort DataFrame by 'rmse' in ascending order
rmse_df = results_df.copy().sort_values(by='rmse', ascending=True)

# Display the sorted DataFrame
rmse_df.head(20)

In [ ]:
# Printing top 5 formulas

top_five_formulas = results_df.head(5)['formula']
print("Top 5 Formulas:")
print(top_five_formulas)


print('---Lowest RMSE Information---')
index_of_min_rmse = results_df['rmse'].idxmin()
print(results_df.loc[index_of_min_rmse, 'formula'])
print('RMSE: ',results_df.loc[index_of_min_rmse, 'rmse'])
print('Pearson R: ', results_df.loc[index_of_min_rmse, 'correlation'])
print('Formula index: ', index_of_min_rmse)

print('\n---Highest Correlation Information---')
index_of_max_corr = results_df['correlation'].idxmax()
print(results_df.loc[index_of_max_corr, 'formula'])
print('RMSE: ',results_df.loc[index_of_max_corr, 'rmse'])
print('Pearson R: ', results_df.loc[index_of_max_corr, 'correlation'])
print('Formula index: ', index_of_max_corr)


In [ ]:
import statsmodels.formula.api as smf
formula_to_test = 0
#----------------------------------------------------------------
results = smf.ols(results_df.loc[formula_to_test, 'formula'], data=df).fit()
# results = smf.ols('percent_change_adascog11 ~ Subiculum*Age*total_hippocampal_csf*frontal', data=data_df).fit()


print(results.summary2())

In [ ]:
from calvin_utils.statistical_measurements import model_diagnostics
model_diagnostics(results)

## Find the Optimal Interaction Subset of Optimal Model by Backward Regression

In [ ]:
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
import itertools


def backward_subsets_loocv_regression(df, formula):
    # Extract outcome_var from the formula
    outcome_var = formula.split(' ~ ')[0].strip()
    
    y_actual, y_predicted = [], []

    for i in range(len(df)):
        index_label = df.index[i]
        x_test = df.iloc[[i]].drop(outcome_var, axis=1)
        x_train = df.drop(index_label).drop(outcome_var, axis=1)
        y_train = df[outcome_var].drop(index_label)

        model_train_df = x_train.join(y_train)
        model_train = smf.ols(formula, data=model_train_df).fit()
        y_actual.append(df[outcome_var].iloc[i])
        y_predicted.append(model_train.predict(x_test).values[0])

    rmse = np.sqrt(mean_squared_error(y_actual, y_predicted))
    corr, _ = pearsonr(y_actual, y_predicted)
    return rmse, corr


def serial_interaction_removal_loocv(df, formula):
    # Split the formula into the left and right side
    left, right = formula.split("~")
    
    # Get all the terms from the right side of the formula
    terms = right.strip().split("*")
    terms = [term.strip() for term in terms]
    
    # Generate all combinations of + and *
    operators_combinations = itertools.product(["+", "*"], repeat=len(terms)-1)
    
    # Generate the formulas
    formulas = []
    for operators in operators_combinations:
        combined_formula = left + " ~ " + terms[0]
        for op, term in zip(operators, terms[1:]):
            combined_formula += " " + op + " " + term
        formulas.append(combined_formula.strip())

    # Run LOOCV on each formula
    loocv_results = []
    for sub_formula in formulas:
        loocv_rmse, loocv_corr = backward_subsets_loocv_regression(df, sub_formula)
        loocv_results.append({
            'formula': sub_formula,
            'rmse': loocv_rmse,
            'correlation': loocv_corr
        })
    
    # Convert results to DataFrame, sort by correlation and reset index
    loocv_results_df = pd.DataFrame(loocv_results)
    loocv_results_df.sort_values(by='correlation', ascending=False, inplace=True)
    loocv_results_df.reset_index(drop=True, inplace=True)

    return loocv_results_df


# Example usage:
# Assuming `df` is your DataFrame, and `formula` is a string like "y ~ a * b * c"
# best_formula = "y ~ a * b * c"
# serial_loocv_df = serial_interaction_removal_loocv(df, best_formula)
# print(serial_loocv_df)

best_formula = results_df.loc[formula_to_test, 'formula']
optimal_interaction_loocv_df = serial_interaction_removal_loocv(df, best_formula)
optimal_interaction_loocv_df.head(20)

# Structural Coefficient Analysis of Optimal Model

In [ ]:
import patsy

def structural_coefficients(model, data):
    # Extracting the predictor names
    predictor_names = model.model.exog_names
    if 'Intercept' in predictor_names:
        predictor_names.remove('Intercept')

    # Calculating the predicted values
    y_predicted = model.predict(data)

    # Creating a temporary dataframe to store interaction terms
    temp_df = pd.DataFrame()
    
    for pred in predictor_names:
        # Check if the predictor is an interaction term
        if ':' in pred:
            # Split the interaction term into its components
            components = pred.split(':')
            # Multiply the components and store the result in the temporary dataframe
            temp_product = data[components[0]]
            for component in components[1:]:
                temp_product *= data[component]
            temp_df[pred] = temp_product
        else:
            temp_df[pred] = data[pred]

    # Calculating the structural coefficients
    structural_coefs = {}
    for pred in predictor_names:
        coef, _ = pearsonr(temp_df[pred], y_predicted)
        structural_coefs[pred] = np.square(coef)

    # Creating a dataframe to store the results
    structural_coefs_df = pd.DataFrame(structural_coefs.items(), columns=['predictor', 'structural_coefficient'])

    # Sorting the dataframe by the structural coefficients in descending order
    structural_coefs_df.sort_values(by='structural_coefficient', ascending=False, inplace=True)
    structural_coefs_df.reset_index(drop=True, inplace=True)

    return structural_coefs_df

In [ ]:
import statsmodels
def structural_coefficients(model: statsmodels.regression.linear_model.RegressionResultsWrapper, 
                            data: pd.DataFrame) -> pd.DataFrame:
    """
    This function calculates the structural coefficients of a linear regression model, along with their 
    associated model coefficients (beta weights). It also computes a 'suppressor index' which indicates 
    the likelihood of a variable being a suppressor variable (high beta weight, near-zero structural coefficient).
    A suppressor index over 10 is a good heuristic for identifying a suppressor variable
    
    Parameters:
    model (statsmodels.regression.linear_model.RegressionResultsWrapper): The fitted linear regression model.
    data (pd.DataFrame): The dataset used in the model.
    
    Returns:
    structural_coefs_df (pd.DataFrame): A dataframe containing the predictors, their structural coefficients,
                                        model coefficients (beta weights), and suppressor index.
                                        
                                        If the sum of the structure coefficients is higher than 1, they are correlated (multicollinear)
    """
    # Calculating the predicted values
    y_predicted = model.predict(data)

    # Creating a temporary dataframe to store interaction terms
    temp_df = pd.DataFrame()
    
    # Calculating the structural coefficients
    structural_coefs = {}
    for pred in model.params.index:
        if pred == 'Intercept':
            continue

        # Check if the predictor is an interaction term
        if ':' in pred:
            # Split the interaction term into its components
            components = pred.split(':')
            # Multiply the components and store the result in the temporary dataframe
            temp_product = data[components[0]]
            for component in components[1:]:
                temp_product *= data[component]
            temp_df[pred] = temp_product
        else:
            temp_df[pred] = data[pred]
        
        coef, _ = pearsonr(temp_df[pred], y_predicted)
        structural_coefs[pred] = np.square(coef)

    # Calculating the model coefficients
    model_coefs = model.params.drop('Intercept')

    # Creating a dataframe to store the results
    structural_coefs_df = pd.DataFrame(list(zip(structural_coefs.keys(), structural_coefs.values(), model_coefs.values)), 
                                       columns=['predictor', 'structural_coefficient', 'model_coefficient'])

    # Adding suppressor index column
    structural_coefs_df['suppressor_index'] = structural_coefs_df['model_coefficient'].abs() / structural_coefs_df['structural_coefficient']

    # Sorting the dataframe by the structural coefficients in descending order
    structural_coefs_df.sort_values(by='structural_coefficient', ascending=False, inplace=True)
    structural_coefs_df.reset_index(drop=True, inplace=True)

    return structural_coefs_df

In [ ]:
squared_structural_coefs_df = structural_coefficients(results, df.copy())
squared_structural_coefs_df.to_csv(os.path.join(out_dir, 'structural_coefficient_analysis.csv'))
print('saved to: ', os.path.join(out_dir, 'structural_coefficient_analysis.csv'))
display(squared_structural_coefs_df)

In [ ]:
#If the sum of the structure coefficients is higher than 1, they are correlated
print(squared_structural_coefs_df.loc[:, 'structural_coefficient'].sum())


In [ ]:
print(results.summary2())


## Check optimal model without LOOCV

In [ ]:
df = df.copy()
no_loocv_results_df = all_subsets_regression_no_loocv(df, outcome_var)
no_loocv_results_df.head(10)



In [ ]:
# Printing top 5 formulas
no_loocv_top_five_formulas = no_loocv_results_df.head(10)['formula']
print("Top 5 Formulas:")
print(no_loocv_top_five_formulas)

In [ ]:
import statsmodels.formula.api as smf
formula_to_test = 0
#----------------------------------------------------------------
results = smf.ols(no_loocv_results_df.loc[formula_to_test, 'formula'], data=df).fit()
print(results.summary2())